In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
from google import genai
import anthropic
from IPython.display import Markdown, display
import re

In [ ]:
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [ ]:
import gradio as gr

In [ ]:
openAi = OpenAI()
gemini = genai.Client(api_key=gemini_api_key)

In [ ]:
def strip_markdown(text):
    text = re.sub(r"\*\*(.*?)\*\*", r"\1", text)
    text = re.sub(r"\*(.*?)\*", r"\1", text)
    text = re.sub(r"`(.*?)`", r"\1", text)
    return text.strip()

In [ ]:
def get_result(text, system_prompt=""):
    result = gemini.models.generate_content(
        model="gemini-2.0-flash",
        contents=text,
        config={
            "system_instruction": [system_prompt],
        },
    )

    return result.candidates[0].content.parts[0].text


def get_result_stream(text, system_prompt=""):
    stream = gemini.models.generate_content_stream(
        model="gemini-2.0-flash",
        contents=text,
        config={
            "system_instruction": [system_prompt],
        },
    )

    output = ""
    for chunk in stream:
        output += chunk.candidates[0].content.parts[0].text or ""
        yield output

In [ ]:
view = gr.Interface(
    fn=get_result,
    inputs=[gr.Textbox(label="Your Message", lines=5)],
    outputs=[gr.Markdown(label="Response")],
    flagging_mode="never",
)
view.launch()